In [ ]:
import numpy as np
from shapely.plotting import plot_polygon

import matplotlib.pyplot as plt
import matplotlib.style as style

style.use(
    "https://raw.githubusercontent.com/dominik-strutz/dotfiles/main/mystyle.mplstyle"
)
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Cmap with only blue and deacreasig alpha
cmap1 = LinearSegmentedColormap.from_list(
    "mycmap",
    [
        "cornflowerblue",
        "mediumblue",
    ],
)

blue_cmap = cmap1
blue_cmap = blue_cmap(np.arange(blue_cmap.N))
blue_cmap[:, -1] = np.linspace(0, 1, blue_cmap.shape[0]) * 0.8
blue_cmap[:, :3] = [0.0, 0.0, 1.0]
blue_cmap = ListedColormap(blue_cmap)


cmap2 = LinearSegmentedColormap.from_list(
    "mycmap",
    [
        "salmon",
        "firebrick",
    ],
)

red_cmap = cmap2
red_cmap = red_cmap(np.arange(red_cmap.N))
red_cmap[:, -1] = np.linspace(0, 1, red_cmap.shape[0]) * 1.0
# red_cmap[:, :3] = [1.0, 0.0, 0.0]
red_cmap = ListedColormap(red_cmap)

In [ ]:
from dased.criteria import RaySensitivity
from dased.optimisation import DASOptimizationProblem, DASArchipelago

In [ ]:
from cdv_setup import (
    topo_data,
    design_space_full,
    obstacles,
    tomo_roi,
)

In [ ]:
cable_properties = {
    "elevation": topo_data,
    "cable_length": 4000,
    "spacing": 10,
    "fixed_points": np.array([[1570.0, 1330.0]]),
    "N_knots": 16,
    "k": 1,
    "signal_decay": 0.4,
}

optimisation_properties = {
    "bounds": [
        [design_space_full.bounds[0], design_space_full.bounds[2]],
        [design_space_full.bounds[1], design_space_full.bounds[3]],
    ],
    "spatial_constraints": design_space_full.envelope.difference(design_space_full),
}

In [ ]:
from interactive_proposal import select_points

##############################################################################
##############################################################################
############### THIS HAS TO BE FIRST PLOT IN THE NOTEBOOK
############### OTHERWISE IT WILL NOT WORK, FOR SOME REASON
##############################################################################
##############################################################################

proposal_points = select_points(
    design_space=design_space_full,
    cable_properties=cable_properties,
    filename="data/optimisation/proposal_rayleigh_length.pkl",
)

##############################################################################
############### IF THIS IS CALLED IT WILL NOT WORK AGAIN
############### BUT IT IS NEEDED TO HAVE LATE PLOTS DISPLAYED
##############################################################################

%matplotlib inline

In [ ]:
def length_criterion(design):
    cl = -design.cable_length
    cl = np.clip(cl, -4000, -1000)  # set min and max length
    return cl

In [ ]:
dx = dy = 20

x_range = (tomo_roi.buffer(50).bounds[0], tomo_roi.buffer(50).bounds[2])
y_range = (tomo_roi.buffer(50).bounds[1], tomo_roi.buffer(50).bounds[3])

n_x = int((x_range[1] - x_range[0]) / dx)
n_y = int((y_range[1] - y_range[0]) / dy)

print(f"n_x = {n_x}, n_y = {n_y}")

rayleigh_criterium = RaySensitivity(
    data_type="rayleigh",
    n_points=(n_x, n_y),
    reference_distance=1_000.0,
    x_range=x_range,
    y_range=y_range,
    roi=tomo_roi,
    criterion="D",
    criterion_kwargs=dict(threshold=1e-4, penalty=10 * np.log(1e-4), normalize=True),
)

criteria_dict = dict(
    Rayleigh_tomo=rayleigh_criterium,
    length_criterion=length_criterion,
)

In [ ]:
udp = DASOptimizationProblem(
    criteria_dict,
    verbose=0,
    **cable_properties,
    **optimisation_properties,
)

In [ ]:
archipelago = DASArchipelago(udp, n_islands=16, population_size=1028, verbose=1)

archipelago.initialize(
    proposal_points=proposal_points,
    perturb_proposal=50.0,
    perturb_knots=25.0,
    corr_len=500,
    corr_str=0.8,
    min_length=1000,
    show_progress=True,
    filename="data/optimisation/initial_population_obstacles_length",
)


In [ ]:
# initial_layouts = archipelago.get_current_layouts()

# lengths = [l.cable_length for l in initial_layouts]

# fig, ax = plt.subplots(figsize=(6, 3))

# ax.hist(
#     lengths, bins=20, color='cornflowerblue', alpha=0.8,
#     edgecolor='black', linewidth=1.2
# )

# ax.set_xlabel('Cable length [m]')
# ax.set_ylabel('Count')
# ax.set_title('Initial cable lengths')

# plt.tight_layout()
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(8, 8), dpi=120)

# plot_polygon(
#     design_space_full, ax=ax, facecolor='none', edgecolor='k', linewidth=2, label='Design Space',
#     add_points=False,
# )

# # Plot fixed points
# fixed_point = cable_properties['fixed_points'][0]
# ax.plot(fixed_point[0], fixed_point[1], 'go', markersize=8, label='Fixed Point')

# # Plot proposal points
# for i, pp in enumerate(proposal_points):
#     pp = np.array(pp)
#     ax.plot(pp[:, 0], pp[:, 1], 'o--', markersize=4,
#            label='Proposal Points' if i == 0 else '')

# # Plot initial layouts
# for layout in initial_layouts[:500]:  # Show fewer layouts for clarity
#     layout.plot(ax=ax, plot_style='line', color='k', alpha=0.3, linewidth=1.0, label='_nolegend_', zorder=-10)
# ax.plot([], [], color='k', alpha=0.3, linewidth=1.0, label='Initial Layouts')

# ax.set_xlabel('Easting (m)')
# ax.set_ylabel('Northing (m)')
# ax.set_title('DAS Layout Optimization - Initial Population')
# ax.grid(True, alpha=0.3)
# ax.set_aspect('equal')

# # Move legend below plot
# ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3, frameon=False)

# plt.tight_layout()
# plt.show()

In [ ]:
archipelago.get_file_info("data/optimisation/optimiser_single_obstacles_length")

In [ ]:
archipelago.optimize(
    n_generations=10000,
    migrate_every=20,
    show_progress=True,
    filename="data/optimisation/optimiser_single_obstacles_length",
)

In [ ]:
archipelago.plot_fitness_history()

In [ ]:
pareto_front = archipelago.get_pareto_front()[1]

In [ ]:
# add four inset plots in lower right two thirds, in each use criteria_dict['rayleigh_tomo_rer'].plot(
#     design=best_layout, ax=ax2, alpha=0.8,) to plot the sensitivit of

In [ ]:
top_layouts, top_fitness = archipelago.get_n_spread_multi(4)


fig, ax = plt.subplots(figsize=(7, 4))

# Main plot: Pareto front
pareto_front = np.array(pareto_front)
pareto_front = pareto_front[pareto_front[:, 0].argsort()]

ax.step(
    -pareto_front[:, 1],
    pareto_front[:, 0],
    where="post",
    color="k",
    linewidth=2,
)

# Mark the top layouts on the pareto front
for i, fitness in enumerate(top_fitness):
    ax.plot(-fitness[1], fitness[0], "ro", markersize=6, zorder=10)

ax.set_xlabel("Cable length [m]")
ax.set_ylabel("Rayleigh D-optimality")

ax.set_ylim(top=1.0)

# Add four inset plots in lower right two thirds
# inset_positions = [
#     [0.45, 0.45, 0.2, 0.2],  # top left inset
#     [0.70, 0.45, 0.2, 0.2],  # top right inset
#     [0.45, 0.2, 0.2, 0.2],   # bottom left inset
#     [0.70, 0.2, 0.2, 0.2]    # bottom right inset
# ]

inset_positions = [
    [0.28, 0.15, 0.2, 0.2],  # top left inset
    [0.14, 0.63, 0.2, 0.2],  # top right inset
    [0.43, 0.42, 0.2, 0.2],  # bottom left inset
    [0.68, 0.53, 0.2, 0.2],  # bottom right inset
]


for i, (layout, fitness) in enumerate(zip(top_layouts, top_fitness)):
    ax_inset = fig.add_axes(inset_positions[i])

    # Plot the sensitivity
    criteria_dict["Rayleigh_tomo"].plot(design=layout, ax=ax_inset, alpha=0.8)

    layout.plot(
        ax=ax_inset,
        plot_style="channels",
        color="k",
        zorder=10,
        width=0.01,
    )

    plot_polygon(
        design_space_full,
        ax=ax_inset,
        facecolor="none",
        edgecolor="tab:grey",
        linestyle="-",
        linewidth=1.0,
        label="viable area",
        add_points=False,
    )

    for obstacle in obstacles.geoms:
        plot_polygon(
            obstacle,
            ax=ax_inset,
            facecolor="none",
            edgecolor="tab:red",
            linestyle="-",
            linewidth=1.0,
            label="viable area",
            add_points=False,
        )

    # add connecting arrow from the main plot to the inset axis not using annotation
    x_main = -fitness[1]
    y_main = fitness[0]

    x_inset = (inset_positions[i][0] + 0.5 * inset_positions[i][2]) * (
        ax.get_xlim()[1] - ax.get_xlim()[0]
    ) + ax.get_xlim()[0]
    y_inset = (inset_positions[i][1] + 0.5 * inset_positions[i][3]) * (
        ax.get_ylim()[1] - ax.get_ylim()[0]
    ) + ax.get_ylim()[0]

    ax.plot([x_main, x_inset], [y_main, y_inset], "k-", alpha=1.0, linewidth=1.0)

    # set facecolor and edgecolor for the inset
    ax_inset.set_facecolor("white")
    # set white box behind title
    ax_inset.set_title(
        f"D-opt: {fitness[0]:.3f} Length: {-fitness[1]:.0f}m",
        fontsize=6,
        backgroundcolor="white",
        pad=6.60,
    )

    ax_inset.set_ylim(1200, 1650)
    ax_inset.set_xlim(500, 1600)
    ax_inset.set_aspect("equal")

    ax_inset.set_xticks([])
    ax_inset.set_yticks([])

fig.savefig(
    "figures/mult_length_pareto_front_obstacles.png", bbox_inches="tight", dpi=300
)
fig.savefig(
    "figures/mult_length_pareto_front_obstacles.pdf", bbox_inches="tight", dpi=300
)

plt.show()

In [ ]:
top_layouts, top_fitness = archipelago.get_n_spread_multi(3)

fitness_history = archipelago.get_fitness_history()
pareto_front = archipelago.get_pareto_front()[1]
best_fitness = archipelago.get_best_multi(method="compromise")[1]

# Define the mosaic layout
mosaic = [
    ["main", "inset0"],
    ["main", "inset1"],
    ["main", "inset2"],
]
fig, axes = plt.subplot_mosaic(
    mosaic, figsize=(10, 4), dpi=130, gridspec_kw={"width_ratios": [3, 1]}
)
ax = axes["main"]

# Use Wistia colormap to show progression over generations
cmap = plt.cm.bone_r

n_gen = len(fitness_history)
generations = [0, 200, 500, 2000]

norm = plt.Normalize(-len(fitness_history) / 4, len(fitness_history))

for idx, gen in enumerate(generations):
    fit_hist = fitness_history[gen]
    # ax.scatter(
    #     -fit_hist[:, 1],
    #     fit_hist[:, 0],
    #     color=[cmap(norm(gen))], edgecolor='k', s=15.
    # )
    sorted_points = fit_hist[np.argsort(fit_hist[:, 1])]
    for i in range(len(sorted_points) - 1):
        ax.plot(
            [-sorted_points[i + 1, 1], -sorted_points[i, 1]],
            [sorted_points[i + 1, 0], sorted_points[i + 1, 0]],
            color=cmap(norm(gen)),
            lw=2.0,
            zorder=-1,
        )
        ax.plot(
            [-sorted_points[i, 1], -sorted_points[i, 1]],
            [sorted_points[i, 0], sorted_points[i + 1, 0]],
            color=cmap(norm(gen)),
            lw=2.0,
            zorder=-1,
        )

# ax.scatter(
#     -pareto_front[:, 1], pareto_front[:, 0], c=[cmap(norm(len(fitness_history)))],
#     label='Best Fitness', s=20, zorder=10
# )top_layouts, top_fitness = archipelago.get_n_spread_multi(4)


sorted_points = pareto_front[np.argsort(pareto_front[:, 1])]
for i in range(len(sorted_points) - 1):
    ax.plot(
        [-sorted_points[i + 1, 1], -sorted_points[i, 1]],
        [sorted_points[i + 1, 0], sorted_points[i + 1, 0]],
        color=cmap(norm(n_gen)),
        lw=2.0,
        zorder=-1,
    )
    ax.plot(
        [-sorted_points[i, 1], -sorted_points[i, 1]],
        [sorted_points[i, 0], sorted_points[i + 1, 0]],
        color=cmap(norm(n_gen)),
        lw=2.0,
        zorder=-1,
    )

legend_elements = []
for gen in generations:
    legend_elements.append(
        plt.Line2D([0], [0], color=cmap(norm(gen)), lw=2, label=f"generation: {gen}")
    )
legend_elements.append(
    plt.Line2D(
        [0],
        [0],
        color=cmap(norm(len(fitness_history))),
        lw=2,
        label="final pareto front",
    )
)

ax.legend(handles=legend_elements, loc="lower right", fontsize=8, frameon=False)

# Plot insets
for i, (layout, fitness) in enumerate(
    zip(reversed(top_layouts), reversed(top_fitness))
):
    ax_inset = axes[f"inset{i}"]
    criteria_dict["Rayleigh_tomo"].plot(design=layout, ax=ax_inset, alpha=0.8)
    layout.plot(
        ax=ax_inset,
        plot_style="channels",
        color="k",
        zorder=10,
        width=0.01,
    )
    plot_polygon(
        design_space_full,
        ax=ax_inset,
        facecolor="none",
        edgecolor="tab:grey",
        linestyle="-",
        linewidth=1.0,
        label="viable area",
        add_points=False,
    )
    for obstacle in obstacles.geoms:
        plot_polygon(
            obstacle,
            ax=ax_inset,
            facecolor="none",
            edgecolor="tab:red",
            linestyle="-",
            linewidth=1.0,
            label="viable area",
            add_points=False,
        )
    x_main = -fitness[1]
    y_main = fitness[0]
    ax.scatter(x_main, y_main, s=60, color="tab:red", zorder=10, marker="o")
    ax_inset.set_facecolor("white")
    ax_inset.set_title(
        f"D-opt: {fitness[0]:.3f} Length: {-fitness[1]:.0f}m",
        fontsize=8,
        backgroundcolor="white",
        pad=6.60,
    )
    ax_inset.set_ylim(1100, 1600)
    ax_inset.set_xlim(500, 1600)
    ax_inset.set_aspect("equal")
    ax_inset.set_xticks([])
    ax_inset.set_yticks([])

ax.set_xlabel("Cable length [m]")
ax.set_ylabel("Rayleigh D-optimality")
ax.set_title("Pareto Front (Best Fitness)")


plt.show()

In [ ]:
endmembers = archipelago.get_endmembers()

In [ ]:
rayleigh_criterium.plot_checkerboard(
    endmembers["Rayleigh_tomo"]["layout"],
    background_velocity=2000.0,
    perturbation=0.05,
    checkerboard_size=4,
    vmin=1800,
    vmax=2200,
    sigma_d=0.002,
    correlation_length=20.0,
    regularization_weight=2.5e-5,
)